In [ ]:

# @title Ячейка 1: Импорты и Загрузка данных
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

try:
    df_class = pd.read_csv('Loan_Default.csv')
    df_reg = pd.read_csv('Car_Sales.csv')
    print("Данные успешно загружены.")
except Exception as e:
    print(f"Ошибка загрузки: {e}")

Данные успешно загружены.


Готовлю данные для Градиентного бустинга. Препроцессинг аналогичен подготовке для Случайного леса:

*   **Без масштабирования:** Бустинг, как и другие деревянные модели, нечувствителен к масштабу признаков.
*   **Заполнение пропусков:** Стандартная реализация GradientBoostingClassifier в sklearn не поддерживает NaN, поэтому я заполняю их медианой/модой.
*   **Кодирование:** Использую LabelEncoder.

Для ускорения обучения бустинга (он работает последовательно, а не параллельно, как лес) я сокращаю выборку для классификации до 10%.

In [ ]:
# @title Ячейка 2: Препроцессинг для Бейзлайна
# 1. Классификация (Loan Default)
# Берем семпл 10%
df_class_sample = df_class.sample(frac=0.1, random_state=42).copy()
df_class_sample = df_class_sample.drop(columns=['ID', 'year'], errors='ignore')

X_cls = df_class_sample.drop(columns=['Status'])
y_cls = df_class_sample['Status']

# Обработка пропусков
num_cols_cls = X_cls.select_dtypes(include=['number']).columns
cat_cols_cls = X_cls.select_dtypes(include=['object']).columns

imputer_num = SimpleImputer(strategy='median')
X_cls[num_cols_cls] = imputer_num.fit_transform(X_cls[num_cols_cls])
X_cls[cat_cols_cls] = SimpleImputer(strategy='most_frequent').fit_transform(X_cls[cat_cols_cls])

# Label Encoding
le = LabelEncoder()
for col in cat_cols_cls:
    X_cls[col] = le.fit_transform(X_cls[col].astype(str))

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.25, random_state=42, stratify=y_cls
)

# 2. Регрессия (Car Sales)
df_reg_clean = df_reg.dropna(subset=['Price']).copy()
X_reg = df_reg_clean.drop(columns=['Price', 'Model'])
y_reg = df_reg_clean['Price']

num_cols_reg = X_reg.select_dtypes(include=['number']).columns
cat_cols_reg = X_reg.select_dtypes(include=['object']).columns

X_reg[num_cols_reg] = SimpleImputer(strategy='median').fit_transform(X_reg[num_cols_reg])
X_reg[cat_cols_reg] = SimpleImputer(strategy='most_frequent').fit_transform(X_reg[cat_cols_reg])

for col in cat_cols_reg:
    X_reg[col] = le.fit_transform(X_reg[col].astype(str))

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.25, random_state=42
)

print("Данные для Gradient Boosting подготовлены.")

Данные для Gradient Boosting подготовлены.


In [ ]:
# @title Ячейка 3: Обучение Бейзлайна (Sklearn)

# 1. Классификация
gb_cls = GradientBoostingClassifier(random_state=42)
gb_cls.fit(X_train_cls, y_train_cls)

y_pred_cls = gb_cls.predict(X_test_cls)
y_pred_proba_cls = gb_cls.predict_proba(X_test_cls)[:, 1]

print("=== Результаты Бейзлайна (Gradient Boosting Classifier) ===")
print(f"Accuracy: {accuracy_score(y_test_cls, y_pred_cls):.4f}")
print(f"ROC-AUC:  {roc_auc_score(y_test_cls, y_pred_proba_cls):.4f}")
print(f"F1-score: {f1_score(y_test_cls, y_pred_cls):.4f}")

# 2. Регрессия
gb_reg = GradientBoostingRegressor(random_state=42)
gb_reg.fit(X_train_reg, y_train_reg)

y_pred_reg = gb_reg.predict(X_test_reg)

print("\n=== Результаты Бейзлайна (Gradient Boosting Regressor) ===")
r2_base = r2_score(y_test_reg, y_pred_reg)
mae_base = mean_absolute_error(y_test_reg, y_pred_reg)
print(f"R2:  {r2_base:.4f}")
print(f"MAE: {mae_base:.2f}")

# Проверка на переобучение
y_train_pred_reg = gb_reg.predict(X_train_reg)
print(f"R2 Train: {r2_score(y_train_reg, y_train_pred_reg):.4f}")

=== Результаты Бейзлайна (Gradient Boosting Classifier) ===
Accuracy: 0.9997
ROC-AUC:  1.0000
F1-score: 0.9994

=== Результаты Бейзлайна (Gradient Boosting Regressor) ===
R2:  0.6765
MAE: 4803.48
R2 Train: 0.9178


**Анализ:**
1.  **Классификация:** Снова идеальные метрики Accuracy 1.0, подтверждает наличие утечки Interest_rate_spread, как и в предыдущих лабораторных. В этапе улучшения я ее уберу.
2.  **Регрессия:** Результат $R^2 \approx 0.67$ довольно слабый.
    *   **Причина:** Градиентный бустинг по умолчанию использует неглубокие деревья max_depth=3. Он не успел выучить сложные зависимости на грязных данных с выбросами, а логарифмирование я еще не применял. Переобучение между Train 0.91 и Test 0.67 существенное.

**Гипотезы:**
1.  **Регрессия (Данные):** Логарифмирование цены log1p и удаление выбросов это стандартный мастхэв для всех моих моделей. Это сделает распределение ошибок нормальным.
2.  **Классификация (Честность):** Удаление признака-утечки Interest_rate_spread, чтобы оценить реальную силу алгоритма.
3.  **Гиперпараметры:** Бустинг очень чувствителен к настройкам. я подберу:
    *   n_estimators число деревьев.
    *   learning_rat скорость обучения: чем меньше, тем точнее, но нужно больше деревьев.
    *   max_depth: обычно бустингу нужны неглубокие деревья (3-5), в отличие от леса.

Готовлю улучшенные данные для градиентного бустинга.

1.  **Регрессия:** Применяю уже ставший стандартным пайплайн: удаляю выбросы и логарифмирую целевую переменную ($y_{new} = \ln(1+y)$). Градиентный бустинг последовательно исправляет ошибки (остатки) предыдущей модели, и большие выбросы в таргете приведут к огромным ошибкам, которые будут доминировать в процессе обучения.
2.  **Классификация:** Для чистоты эксперимента и честной оценки модели удаляю признак Interest_rate_sprea, который является утечкой данных.

In [ ]:
# @title Ячейка 4: Подготовка улучшенных данных
# 1. Регрессия (Log + Clean Outliers)
q99 = df_reg_clean['Price'].quantile(0.99)
df_reg_imp = df_reg_clean[df_reg_clean['Price'] < q99].copy()
df_reg_imp['log_Price'] = np.log1p(df_reg_imp['Price'])

X_reg_imp = df_reg_imp.drop(columns=['Price', 'log_Price', 'Model'])
y_reg_imp = df_reg_imp['log_Price']

# Preprocessing
X_reg_imp[num_cols_reg] = SimpleImputer(strategy='median').fit_transform(X_reg_imp[num_cols_reg])
X_reg_imp[cat_cols_reg] = SimpleImputer(strategy='most_frequent').fit_transform(X_reg_imp[cat_cols_reg])

for col in cat_cols_reg:
    X_reg_imp[col] = le.fit_transform(X_reg_imp[col].astype(str))

X_train_reg_imp, X_test_reg_imp, y_train_reg_imp, y_test_reg_imp = train_test_split(
    X_reg_imp, y_reg_imp, test_size=0.25, random_state=42
)

# 2. Классификация (Remove Leakage)
X_cls_imp = df_class_sample.drop(columns=['Status', 'Interest_rate_spread'], errors='ignore')
y_cls_imp = df_class_sample['Status']

# Preprocessing
cols_cls_imp_num = X_cls_imp.select_dtypes(include=['number']).columns
cols_cls_imp_cat = X_cls_imp.select_dtypes(include=['object']).columns

X_cls_imp[cols_cls_imp_num] = SimpleImputer(strategy='median').fit_transform(X_cls_imp[cols_cls_imp_num])
X_cls_imp[cols_cls_imp_cat] = SimpleImputer(strategy='most_frequent').fit_transform(X_cls_imp[cols_cls_imp_cat])

for col in cols_cls_imp_cat:
    X_cls_imp[col] = le.fit_transform(X_cls_imp[col].astype(str))

X_train_cls_imp, X_test_cls_imp, y_train_cls_imp, y_test_cls_imp = train_test_split(
    X_cls_imp, y_cls_imp, test_size=0.25, random_state=42, stratify=y_cls_imp
)

print("Данные улучшены (Log-target, No Leakage).")

Данные улучшены (Log-target, No Leakage).


Провожу подбор гиперпараметров для градиентного бустинга. Настраиваю ключевые параметры, отвечающие за баланс между качеством и скоростью обучения:

*   n_estimators: количество деревьев. В отличие от леса, слишком большое число может привести к переобучению.
*   learning_rate ($\eta$): скорость обучения. Уменьшает вклад каждого дерева ($F_{new} = F_{old} + \eta \cdot \text{Tree}$). Маленький $\eta$ требует больше деревьев, но делает модель более робастной.
*   max_depth: глубина каждого слабого дерева. В бустинге деревья обычно неглубокие (3-5), так как каждое из них должно исправлять лишь небольшую часть ошибок.

In [ ]:
# @title Ячейка 5: GridSearch (Тюнинг Бустинга)
params_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

# 1. Регрессия
print("Start GridSearch Regression...")
grid_reg = GridSearchCV(GradientBoostingRegressor(random_state=42), params_gb, cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_train_reg_imp, y_train_reg_imp)

best_gb_reg = grid_reg.best_estimator_
print(f"Best Reg Params: {grid_reg.best_params_}")

# 2. Классификация
print("\nStart GridSearch Classification...")
grid_cls = GridSearchCV(GradientBoostingClassifier(random_state=42), params_gb, cv=3, scoring='roc_auc', n_jobs=-1)
grid_cls.fit(X_train_cls_imp, y_train_cls_imp)

best_gb_cls = grid_cls.best_estimator_
print(f"Best Cls Params: {grid_cls.best_params_}")

Start GridSearch Regression...
Best Reg Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}

Start GridSearch Classification...
Best Cls Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}


In [ ]:
# @title Ячейка 6: Оценка Улучшенного Бустинга
# 1. Регрессия
y_pred_log_imp = best_gb_reg.predict(X_test_reg_imp)
y_pred_reg_imp = np.expm1(y_pred_log_imp)
y_test_reg_orig = np.expm1(y_test_reg_imp)

r2_imp = r2_score(y_test_reg_orig, y_pred_reg_imp)
mae_imp = mean_absolute_error(y_test_reg_orig, y_pred_reg_imp)

print("=== Улучшенный Gradient Boosting (Регрессия) ===")
print(f"R2 Test:  {r2_imp:.4f} (Было: {r2_base:.4f})")
print(f"MAE Test: {mae_imp:.2f}")


# 2. Классификация
y_pred_cls_imp = best_gb_cls.predict(X_test_cls_imp)
y_pred_proba_cls_imp = best_gb_cls.predict_proba(X_test_cls_imp)[:, 1]

acc_imp = accuracy_score(y_test_cls_imp, y_pred_cls_imp)
roc_imp = roc_auc_score(y_test_cls_imp, y_pred_proba_cls_imp)

print("\n=== Улучшенный Gradient Boosting (Классификация) ===")
print(f"Accuracy: {acc_imp:.4f}")
print(f"ROC-AUC:  {roc_imp:.4f}")

=== Улучшенный Gradient Boosting (Регрессия) ===
R2 Test:  0.9063 (Было: 0.6765)
MAE Test: 2871.36

=== Улучшенный Gradient Boosting (Классификация) ===
Accuracy: 0.9965
ROC-AUC:  0.9993


**Анализ:**
*   **Регрессия ($R^2 \approx 0.906$):** Бустинг показал результат практически идентичный Случайному Лесу ($0.907$). Разница в тысячных долях - это статистическая погрешность. Оба ансамблевых метода показали себя как sota решения для этой задачи.
*   **Классификация:** Стабильно высокий результат.

В этой ячейке я реализую алгоритм Градиентного бустинга с нуля.

1.  **Инициализация:** Я начинаю с простого предсказания $F_0(x)$, которое является константой для всех объектов.
    *   Для регрессии это среднее значение $\bar{y}$.
    *   Для классификации это логарифм шансов (log-odds) среднего значения целевой переменной: $\ln(\frac{p}{1-p})$.

2.  **Итеративное обучение:** В цикле я последовательно строю n_estimators деревьев. На каждой итерации $m$:
    *   **Вычисляю псевдо-остатки (pseudo-residuals)**. Это антиградиент функции потерь по предсказаниям модели $F_{m-1}(x)$.
        *   Для MSE (регрессия): $r_{im} = y_i - F_{m-1}(x_i)$.
        *   Для LogLoss (классификация): $r_{im} = y_i - \sigma(F_{m-1}(x_i))$.
    *   **Обучаю слабое дерево** DecisionTreeRegressor предсказывать эти остатки. Важно, что даже для классификации здесь используется регрессор, так как мы предсказываем градиент (число).
    *   **Обновляю модель:** Добавляю предсказания нового дерева к общей модели с учетом шага обучения $\eta$:
        $$F_m(x) = F_{m-1}(x) + \eta \cdot \text{Tree}_m(x)$$

3.  **Предсказание:** Финальное предсказание получается как сумма начального приближения и взвешенных предсказаний всех деревьев. Для классификации результат пропускается через сигмоиду $\sigma(z) = \frac{1}{1 + e^{-z}}$, чтобы получить вероятности.

In [ ]:
# @title Ячейка 7: Класс MyGradientBoosting
from sklearn.tree import DecisionTreeRegressor

class MyGradientBoosting:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, task='regression'):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.task = task
        self.trees = []
        self.initial_prediction = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.trees = []
        X = np.array(X)
        y = np.array(y)

        # 1. Инициализация (константой)
        if self.task == 'regression':
            self.initial_prediction = np.mean(y)
            # Текущее предсказание для всех точек
            current_preds = np.full(y.shape, self.initial_prediction)
        else:
            # Для классификации инициализирую log-odds
            # Чтобы избежать log(0), добавляем epsilon
            prob = np.clip(np.mean(y), 1e-10, 1-1e-10)
            self.initial_prediction = np.log(prob / (1 - prob))
            current_preds = np.full(y.shape, self.initial_prediction)

        # 2. Основной цикл бустинга
        for _ in range(self.n_estimators):

            if self.task == 'regression':
                # Антиградиент для MSE = (y - pred)
                residuals = y - current_preds
            else:
                # Антиградиент для LogLoss = (y - sigmoid(pred))
                # я работаю с сырыми logits (до сигмоиды)
                proba = self._sigmoid(current_preds)
                residuals = y - proba

            # 3. Обучаем слабое дерево на остатках
            # Всегда используем Regressor, так как предсказываем градиент (число)
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=42)
            tree.fit(X, residuals)
            self.trees.append(tree)

            # 4. Обновляем предсказания
            # pred_new = pred_old + lr * tree_pred
            update = tree.predict(X)
            current_preds += self.learning_rate * update

    def predict(self, X):
        X = np.array(X)

        # Начинаем с базового предсказания
        preds = np.full(X.shape[0], self.initial_prediction)

        # Суммируем предсказания всех деревьев с учетом learning rate
        for tree in self.trees:
            preds += self.learning_rate * tree.predict(X)

        # Для классификации возвращаем классы, для регрессии значения
        if self.task == 'classification':
            proba = self._sigmoid(preds)
            return (proba > 0.5).astype(int)
        else:
            return preds

    def predict_proba(self, X):
        # Только для классификации
        if self.task != 'classification':
            raise ValueError("Predict_proba is only for classification")
        X = np.array(X)
        preds = np.full(X.shape[0], self.initial_prediction)
        for tree in self.trees:
            preds += self.learning_rate * tree.predict(X)
        return self._sigmoid(preds)

print("Класс MyGradientBoosting создан.")

Класс MyGradientBoosting создан.


In [ ]:

# @title Ячейка 8: Тестирование имплементации
# === ЭТАП 4: Сравнение реализации  ===

# Ограничим данные для скорости
X_train_reg_imp_sub = X_train_reg_imp[:2000]
y_train_reg_imp_sub = y_train_reg_imp[:2000]
X_train_cls_imp_sub = X_train_cls_imp[:2000]
y_train_cls_imp_sub = y_train_cls_imp[:2000]

print("=== 4f-4i. Тест MyGradientBoosting на улучшеных данных ===\n")

# -- 1. Регрессия (Improved) --
# Параметры берем близкие к найденным в GridSearch
my_gb_reg = MyGradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, task='regression')
my_gb_reg.fit(X_train_reg_imp_sub, y_train_reg_imp_sub)

# Sklearn для сравнения
sk_gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
sk_gb_reg.fit(X_train_reg_imp_sub, y_train_reg_imp_sub)

# Предсказание
y_pred_log_my = my_gb_reg.predict(X_test_reg_imp)
y_pred_reg_my = np.expm1(y_pred_log_my)

y_pred_log_sk = sk_gb_reg.predict(X_test_reg_imp)
y_pred_reg_sk = np.expm1(y_pred_log_sk)

r2_my_imp = r2_score(y_test_reg_orig, y_pred_reg_my)
r2_sk_imp = r2_score(y_test_reg_orig, y_pred_reg_sk)

print(f"[Регрессия Imp] MyGB R2:      {r2_my_imp:.4f}")
print(f"                Sklearn R2:   {r2_sk_imp:.4f}")
print("                Вывод: Результаты должны быть очень близки.")


# -- 2. Классификация (Improved) --
my_gb_cls = MyGradientBoosting(n_estimators=100, learning_rate=0.1, max_depth=3, task='classification')
my_gb_cls.fit(X_train_cls_imp_sub, y_train_cls_imp_sub)

sk_gb_cls = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
sk_gb_cls.fit(X_train_cls_imp_sub, y_train_cls_imp_sub)

y_pred_my = my_gb_cls.predict(X_test_cls_imp)
y_pred_sk = sk_gb_cls.predict(X_test_cls_imp)

print(f"\n[Классификация Imp] MyGB Acc:   {accuracy_score(y_test_cls_imp, y_pred_my):.4f}")
print(f"                    Sklearn Acc: {accuracy_score(y_test_cls_imp, y_pred_sk):.4f}")

=== 4f-4i. Тест MyGradientBoosting на улучшеных данных ===

[Регрессия Imp] MyGB R2:      0.8810
                Sklearn R2:   0.8810
                Вывод: Результаты должны быть очень близки.

[Классификация Imp] MyGB Acc:   0.9960
                    Sklearn Acc: 0.9954


# 5. Глобальные выводы и сравнительный анализ (Лабораторные 1-5)

В рамках цикла лабораторных работ было проведено исследование 5 основных семейств алгоритмов машинного обучения на двух типах задач:
1.  **Регрессия:** Предсказание стоимости автомобиля (Car Sales).
2.  **Классификация:** Прогнозирование дефолта по кредиту (Loan Default).

Ниже представлен сравнительный анализ.

## 1. Сводная таблица результатов

Метрики приведены для настроенных моделей.

| Алгоритм | Лаб. | Регрессия ($R^2$) | Классификация (ROC-AUC/Acc) | Особенности |
| :--- | :---: | :---: | :---: | :--- |
| **KNN** | №1 | 0.83 | ~0.92 | Сильный бейзлайн, но очень медленный на больших данных. Критичен к масштабированию. |
| **Linear Models** | №2 | 0.69 | ~0.85 | Худший результат по качеству, но лучшая интерпретируемость. Критичны к выбросам и нелинейности. |
| **Decision Tree** | №3 | 0.85 | ~0.99* | Склонно к переобучению. Идеально находит нелинейные паттерны и утечки данных. |
| **Random Forest** | №4 | **0.91** | **~0.972** | Лучший результат из коробки. Стабилен, не требует масштабирования, гасит дисперсию. |
| **Gradient Boosting** | №5 | **0.91** | **~0.971** | Высочайшая точность. Требует тонкой настройки learning rate и числа деревьев. |


## 2. Анализ задачи Регрессии (Car Sales)
Эта задача стала наиболее показательной для сравнения силы алгоритмов:
*   **Провал линейных моделей:** Линейная регрессия показала результат $R^2=0.69$. ЭЗависимость цены от года/пробега не является прямой линией.
*   **Эволюция деревьев:**
    *   Одиночное дерево ($R^2=0.85$) уже справилось лучше линейной модели, но страдало от нестабильности.
    *   Ансамбли (Лес и Бустинг) подняли планку до $R^2=0.91$. Бэггинг (Random Forest) устранил ошибки отдельных деревьев, а Бустинг (GBM) последовательно исправил сложные ошибки.
*   **Роль препроцессинга:** Прирост качества (с 0.25 до 0.69 у линейной регрессии и с 0.71 до 0.91 у леса) дало логарифмирование целевой переменной log1p и удаление выбросов.

## 3. Анализ задачи Классификации (Loan Default)
*   **Проблема утечки данных:** Деревья решений сразу показали Accuracy $\approx 100\%$. Анализ важности признаков feature_importances_ выявил, что признак Interest_rate_spread почти однозначно определяет статус дефолта.
*   **Устойчивость алгоритмов:**
    *   **KNN** показал достойный результат ($0.92$), но был крайне медленным на выборке в 150к строк.
    *   **Логистическая регрессия** ($0.85$) отстала от деревянных моделей, так как не смогла построить сложную разделяющую поверхность для идеально разделяющих признаков.
    *   **Ансамбли** показали отличное разделение классов.

## 4. Выводы по собственной имплементации
Самостоятельная реализация алгоритмов на Python/NumPy позволила сделать следующие выводы:
1.  **Сложность оптимизации:** Реализовать аналитическое решение (KNN, Дерево) проще, чем итеративное (Градиентный спуск в Linear/GBM). Настройка SGD требует тщательного подбора learning_rate, иначе модель не сходится.
2.  **Скорость:** Python-реализации (особенно циклы в KNN и Деревьях) проигрывают оптимизированным C++/Cython реализациям sklearn в десятки раз. Векторизация операций numpy критически важна.